In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df_train = pd.read_csv('z_train.csv')
df_test = pd.read_csv('z_test.csv')

In [5]:
df_train_copy = df_train.copy()
df_test_copy = df_test.copy()

In [6]:
df_train_copy[df_train_copy.target==1].shape

(3824, 14)

In [7]:
df_train_copy[df_train_copy.target==0].shape

(11502, 14)

In [8]:
df_test_copy.drop(['enrollee_id', 'city','company_size'], axis=1 , inplace = True)

In [9]:
df_train_copy.drop(['enrollee_id', 'city','company_size'], axis=1 , inplace = True)


In [10]:
df_train_copy.dropna(inplace=True)

In [11]:
df_train_copy.shape

(7405, 11)

In [12]:
df_train.shape

(15326, 14)

In [13]:
def cambiar_mayor(num):
    num= str(num)
    if '>' in num:
        num = num.replace('>','')
        return num
    
    if '<' in num:
        num = num.replace('<1','0')
        return num
    
    if 'nan' in num:
        num = num.replace('nan','')
        return num
    
    else:
        return num

In [14]:
df_train_copy.experience = df_train_copy.experience.apply(cambiar_mayor)

In [15]:
df_train_copy['experience'] = pd.to_numeric(df_train_copy.experience)

In [16]:
df_test_copy.experience = df_train_copy.experience.apply(cambiar_mayor)
df_test_copy['experience'] = pd.to_numeric(df_train_copy.experience)

In [17]:
def cambiar_num(num):
    num= str(num)
    if '>4' in num:
        num = num.replace('>4','5')
        return num
    
    if 'never' in num:
        num = num.replace('never','0')
        return num
    
    if 'nan' in num:
        num = num.replace('nan','')
        return num
    
    else:
        return num

In [18]:
df_train_copy.last_new_job = df_train_copy.last_new_job.apply(cambiar_num)

In [19]:
df_train_copy['last_new_job'] = pd.to_numeric(df_train_copy.last_new_job)

In [20]:
df_test_copy.last_new_job = df_train_copy.last_new_job.apply(cambiar_num)
df_test_copy['last_new_job'] = pd.to_numeric(df_train_copy.last_new_job)

In [21]:
df_train_copy.columns

Index(['city_development_index', 'gender', 'relevent_experience',
       'enrolled_university', 'education_level', 'major_discipline',
       'experience', 'company_type', 'last_new_job', 'training_hours',
       'target'],
      dtype='object')

In [22]:
df_train_copy.experience.value_counts()

20    1614
5      480
10     453
6      447
9      442
7      412
4      399
3      364
15     333
8      317
11     314
14     284
2      253
16     251
12     220
13     184
17     170
19     146
18     134
1      107
0       81
Name: experience, dtype: int64

In [23]:
pip install feature_engine

Note: you may need to restart the kernel to use updated packages.


In [24]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import RobustScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split
from feature_engine import imputation as imp
from feature_engine import encoding as enc
from feature_engine.wrappers import SklearnTransformerWrapper
from feature_engine.encoding import OneHotEncoder
from feature_engine import imputation as mdi


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

from feature_engine.wrappers import SklearnTransformerWrapper


from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import sklearn.metrics

In [25]:
X_train, X_test, y_train, y_test = train_test_split(df_train_copy.drop(columns = ['target']), df_train_copy.target, test_size = 0.2, random_state = 830)

In [26]:
df_train_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7405 entries, 1 to 15323
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city_development_index  7405 non-null   float64
 1   gender                  7405 non-null   object 
 2   relevent_experience     7405 non-null   object 
 3   enrolled_university     7405 non-null   object 
 4   education_level         7405 non-null   object 
 5   major_discipline        7405 non-null   object 
 6   experience              7405 non-null   int64  
 7   company_type            7405 non-null   object 
 8   last_new_job            7405 non-null   int64  
 9   training_hours          7405 non-null   int64  
 10  target                  7405 non-null   float64
dtypes: float64(2), int64(3), object(6)
memory usage: 952.3+ KB


In [27]:
df_train_copy.training_hours

1         47
6         18
7         46
10       108
11        23
        ... 
15316     51
15317     36
15318     23
15320     25
15323     44
Name: training_hours, Length: 7405, dtype: int64

In [28]:
pipeline_gilda = Pipeline([('mean_imputation', mdi.MeanMedianImputer(imputation_method = 'median',
                                                                             variables = ['experience', 'last_new_job','training_hours'])),
                                   ('mode_imputation', mdi.CategoricalImputer(imputation_method = 'missing',
                                    variables = ['enrolled_university', 'education_level', 'major_discipline','relevent_experience','gender','company_type'])),
                                   ('escalar', SklearnTransformerWrapper(StandardScaler(), variables=['experience', 'last_new_job','training_hours'])),
                                   ('one_hot', enc.OneHotEncoder(variables = ['enrolled_university', 'education_level', 'major_discipline','relevent_experience','gender','company_type'])),                         
                                   ], verbose = True)
pipeline_gilda.fit(X_train, y_train)

[Pipeline] ... (step 1 of 4) Processing mean_imputation, total=   0.0s
[Pipeline] ... (step 2 of 4) Processing mode_imputation, total=   0.0s
[Pipeline] ........... (step 3 of 4) Processing escalar, total=   0.0s
[Pipeline] ........... (step 4 of 4) Processing one_hot, total=   0.0s


Pipeline(steps=[('mean_imputation',
                 MeanMedianImputer(variables=['experience', 'last_new_job',
                                              'training_hours'])),
                ('mode_imputation',
                 CategoricalImputer(variables=['enrolled_university',
                                               'education_level',
                                               'major_discipline',
                                               'relevent_experience', 'gender',
                                               'company_type'])),
                ('escalar',
                 SklearnTransformerWrapper(transformer=StandardScaler(),
                                           variables=['experience',
                                                      'last_new_job',
                                                      'training_hours'])),
                ('one_hot',
                 OneHotEncoder(variables=['enrolled_university',
                             

In [29]:
test_1 = pipeline_gilda.transform(df_test_copy)

In [30]:
test_1.shape

(3832, 27)

In [31]:
data_1 = pipeline_gilda.transform(X_train)

In [32]:
data_1.shape

(5924, 27)

In [33]:
X_train.head(5)

,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_type,last_new_job,training_hours
2712,0.920,Male,Has relevent experience,no_enrollment,Graduate,Arts,15,Funded Startup,5,102
12366,0.910,Female,No relevent experience,no_enrollment,Graduate,Arts,3,Pvt Ltd,2,3
3853,0.926,Male,Has relevent experience,no_enrollment,Graduate,STEM,9,Pvt Ltd,0,122
4495,0.910,Male,Has relevent experience,no_enrollment,Graduate,STEM,10,Pvt Ltd,1,40
641,0.802,Male,Has relevent experience,no_enrollment,Graduate,STEM,4,Pvt Ltd,1,31


In [34]:
data_1.head(5)

,city_development_index,experience,last_new_job,training_hours,enrolled_university_no_enrollment,enrolled_university_Full time course,enrolled_university_Part time course,education_level_Graduate,education_level_Masters,education_level_Phd,...,relevent_experience_No relevent experience,gender_Male,gender_Female,gender_Other,company_type_Funded Startup,company_type_Pvt Ltd,company_type_Public Sector,company_type_Early Stage Startup,company_type_NGO,company_type_Other
2712,0.920,0.578543,1.605797,0.595879,1,0,0,1,0,0,...,0,1,0,0,1,0,0,0,0,0
12366,0.910,-1.343031,-0.188947,-1.032405,1,0,0,1,0,0,...,1,0,1,0,0,1,0,0,0,0
3853,0.926,-0.382244,-1.385443,0.924825,1,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
4495,0.910,-0.222113,-0.787195,-0.423854,1,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
641,0.802,-1.182900,-0.787195,-0.571880,1,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0


In [35]:
data_1.columns

Index(['city_development_index', 'experience', 'last_new_job',
       'training_hours', 'enrolled_university_no_enrollment',
       'enrolled_university_Full time course',
       'enrolled_university_Part time course', 'education_level_Graduate',
       'education_level_Masters', 'education_level_Phd',
       'major_discipline_Arts', 'major_discipline_STEM',
       'major_discipline_Other', 'major_discipline_Humanities',
       'major_discipline_Business Degree', 'major_discipline_No Major',
       'relevent_experience_Has relevent experience',
       'relevent_experience_No relevent experience', 'gender_Male',
       'gender_Female', 'gender_Other', 'company_type_Funded Startup',
       'company_type_Pvt Ltd', 'company_type_Public Sector',
       'company_type_Early Stage Startup', 'company_type_NGO',
       'company_type_Other'],
      dtype='object')

In [36]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import tree

In [37]:
from sklearn.ensemble import RandomForestClassifier

In [38]:
grid_forestrest = RandomForestClassifier()
params = {'max_depth': [4, 7, 10, 15], 'n_estimators':[21, 32, 59, 78]}
grid_forest = GridSearchCV(grid_forestrest, params).fit(data_1, y_train)

In [39]:
grid_forest.best_estimator_

RandomForestClassifier(max_depth=7, n_estimators=32)

In [40]:
best_forest = RandomForestClassifier(max_depth=10, n_estimators=59).fit(data_1, y_train)

In [41]:
pd.DataFrame([best_forest.feature_importances_], columns = best_forest.feature_names_in_).T

,0
city_development_index,0.537338
experience,0.123796
last_new_job,0.053632
training_hours,0.107324
enrolled_university_no_enrollment,0.009371
enrolled_university_Full time course,0.016756
enrolled_university_Part time course,0.007740
education_level_Graduate,0.011346
education_level_Masters,0.011078
education_level_Phd,0.005274


In [42]:
pred_y = best_forest.predict(data_1)

In [43]:
pred_y

array([0., 0., 0., ..., 0., 0., 0.])

In [44]:
metrics.roc_auc_score(y_train, pred_y)

0.7398078401926709

In [45]:
from sklearn.metrics import roc_auc_score

In [46]:
metrics.accuracy_score(y_train, pred_y)

0.8946657663740716

In [47]:
test_2 = pipeline_gilda.transform(X_test)

In [48]:
pred_y_test = best_forest.predict(test_2)

In [49]:
metrics.roc_auc_score(y_test, pred_y_test)

0.6669102170906248

In [50]:
df_test_copy = df_test.copy(deep=False)

In [51]:
prediccion = df_test_copy.copy(deep=False)

In [52]:
prediccion.drop(['city', 'city_development_index', 'gender',
       'relevent_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'last_new_job', 'training_hours'],axis=1, inplace=True)

In [53]:
pred_y_test = best_forest.predict_proba(test_1)

In [54]:
 best_forest.predict(test_1)

array([0., 0., 0., ..., 0., 0., 0.])

In [55]:
pred_y_test[:-1]

array([[0.82260266, 0.17739734],
       [0.89824154, 0.10175846],
       [0.85059994, 0.14940006],
       ...,
       [0.92987183, 0.07012817],
       [0.90788634, 0.09211366],
       [0.95592714, 0.04407286]])

In [56]:
datos=[]
for x in pred_y_test:
    datos.append(x[1])

In [57]:
target = pd.DataFrame(datos)

In [58]:
prediccion['target'] = target

In [59]:
prediccion

,enrollee_id,target
0,23603,0.177397
1,22499,0.101758
2,10465,0.149400
3,8293,0.109397
4,4246,0.097448
...,...,...
3827,8880,0.104415
3828,7886,0.070128
3829,12279,0.092114
3830,5326,0.044073


In [60]:
prediccion.to_csv('prediccion1_gilda.csv', index=False)